In [ ]:
import os
import tkinter as tk
from tkinter import messagebox
import cv2
import webbrowser
import subprocess
from geopy.geocoders import Nominatim
import smtplib
import pywhatkit as pwk
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import load_tools, AgentType, initialize_agent
import mediapipe as mp


os.environ["GOOGLE_API_KEY"] = "Google API key"
os.environ["SERPAPI_API_KEY"] = "Serpapi key"
llm = ChatGoogleGenerativeAI(model="gemini-pro")
myserptool = load_tools(tool_names=["serpapi"])
mygoogleai = initialize_agent(
    llm=llm,
    tools=myserptool,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

def apply_sketch(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inverted_gray_frame = 255 - gray_frame
    blurred_frame = cv2.GaussianBlur(inverted_gray_frame, (21, 21), 0)
    inverted_blurred_frame = 255 - blurred_frame
    sketch = cv2.divide(gray_frame, inverted_blurred_frame, scale=256.0)
    return sketch

def start_sketch_app():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame.")
            break
        sketch = apply_sketch(frame)
        cv2.imshow('Original', frame)
        cv2.imshow('Sketch Effect', sketch)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def send_email():
    email_sender = '12345ujjwalpratap@gmail.com'
    email_password = input("Enter your Password: ")
    email_receiver = '12345masterups@gmail.com'
    msg = "How are you ujjwal?"
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_sender, email_password)
        print("Login Success")
        server.sendmail(email_sender, email_receiver, msg)
        print("Email has been sent successfully")
        messagebox.showinfo("Success", "Email has been sent successfully")
        server.quit()
    except Exception as e:
        print("Error:", e)
        messagebox.showerror("Error", f"An error occurred: {e}")

def get_location_coordinates():
    location = input("Enter the location to get coordinates: ")
    try:
        geolocator = Nominatim(user_agent="GetLoc")
        get_location = geolocator.geocode(location)
        messagebox.showinfo("Location Details", f"Address: {get_location.address}\nLatitude: {get_location.latitude}\nLongitude: {get_location.longitude}")
    except Exception as e:
        print("Error:", e)
        messagebox.showerror("Error", f"An error occurred: {e}")

def open_calculator():
    subprocess.Popen('calc.exe')

def open_camera():
    subprocess.Popen('start microsoft.windows.camera:', shell=True)

def open_settings():
    subprocess.Popen('start ms-settings:', shell=True)

def open_gmail():
    webbrowser.open('https://mail.google.com')

def open_whatsapp():
    webbrowser.open('https://web.whatsapp.com')

def search_word():
    word = input("Type the word to search: ")
    try:
        pwk.search(word)
    except Exception as e:
        print("Error occurred:", e)
        messagebox.showerror("Error", f"An error occurred: {e}")

def button1_action():
    start_sketch_app()

def button2_action():
    send_email()

def button3_action():
    get_location_coordinates()

def button4_action():
    window = tk.Toplevel(root)
    window.title("Open Applications")
    btn_gmail = tk.Button(window, text="Open Gmail", command=open_gmail)
    btn_gmail.pack(pady=10)
    btn_camera = tk.Button(window, text="Open Camera", command=open_camera)
    btn_camera.pack(pady=10)
    btn_settings = tk.Button(window, text="Open Settings", command=open_settings)
    btn_settings.pack(pady=10)
    btn_whatsapp = tk.Button(window, text="Open WhatsApp", command=open_whatsapp)
    btn_whatsapp.pack(pady=10)

def button5_action():
    search_word()

def button6_action():
    mygoogleai.run("today date ")

def button7_action():
    detect_gesture()

def detect_gesture():
    mp_drawing = mp.solutions.drawing_utils
    mp_holistic = mp.solutions.holistic

    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                continue

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(frame_rgb)

            if results.left_hand_landmarks or results.right_hand_landmarks:
                if results.left_hand_landmarks:
                    thumb_tip = results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP]
                    index_tip = results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP]

                    if thumb_tip and index_tip and thumb_tip.y < index_tip.y:
                        cv2.putText(frame, "done", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                if results.right_hand_landmarks:
                    thumb_tip = results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.THUMB_TIP]
                    index_tip = results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP]

                    if thumb_tip and index_tip and thumb_tip.y < index_tip.y:
                        cv2.putText(frame, "done", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                if results.left_hand_landmarks:
                    wrist = results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST]
                    pinky_tip = results.left_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP]

                    if wrist and pinky_tip and pinky_tip.y < wrist.y:
                        cv2.putText(frame, "Hey", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                if results.right_hand_landmarks:
                    wrist = results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST]
                    pinky_tip = results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.PINKY_TIP]

                    if wrist and pinky_tip and pinky_tip.y < wrist.y:
                        cv2.putText(frame, "Hey", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
            mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            cv2.imshow('MediaPipe Holistic', frame)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

root = tk.Tk()
root.title("Multi-function Application")

buttons_frame = tk.Frame(root)
buttons_frame.pack(padx=10, pady=10)

btn1 = tk.Button(buttons_frame, text="Start Sketch App", command=button1_action)
btn1.grid(row=0, column=0, padx=10, pady=5)

btn2 = tk.Button(buttons_frame, text="Send Email", command=button2_action)
btn2.grid(row=0, column=1, padx=10, pady=5)

btn3 = tk.Button(buttons_frame, text="Get Location Coordinates", command=button3_action)
btn3.grid(row=1, column=0, padx=10, pady=5)

btn4 = tk.Button(buttons_frame, text="Open Applications", command=button4_action)
btn4.grid(row=1, column=1, padx=10, pady=5)

btn5 = tk.Button(buttons_frame, text="Search Word", command=button5_action)
btn5.grid(row=2, column=0, columnspan=2, padx=10, pady=5)

btn6 = tk.Button(buttons_frame, text="Run AI Agent", command=button6_action)
btn6.grid(row=3, column=0, columnspan=2, padx=10, pady=5)

btn7 = tk.Button(buttons_frame, text="Detect Gesture", command=button7_action)
btn7.grid(row=4, column=0, columnspan=2, padx=10, pady=5)

root.mainloop()

C:\Users\Rajan\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
